<a href="https://colab.research.google.com/github/Mushfique12/IGTE/blob/main/pinns_magnetostatic_poisson_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install -U "ray[tune]"

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 54.5 MB 178 kB/s 
     |████████████████████████████████| 4.1 MB 62.7 MB/s 
     |████████████████████████████████| 8.8 MB 62.2 MB/s 
     |████████████████████████████████| 125 kB 68.4 MB/s 
     |████████████████████████████████| 466 kB 74.7 MB/s 
  Attempting uninstall: grpcio
    Found existing installation: grpcio 1.47.0
    Uninstalling grpcio-1.47.0:
      Successfully uninstalled grpcio-1.47.0


In [ ]:
from typing import Callable

import matplotlib.pyplot as plt
import torch
from scipy.integrate import solve_ivp
from torch import nn
import numpy as np
import scipy.io
import time
import urllib

from ray import tune
from ray.tune import CLIReporter
from ray.tune.schedulers import ASHAScheduler

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [ ]:
# Checking for GPU
if torch.cuda.is_available():
  tensor = torch.rand(3, 4)
  tensor = tensor.to('cuda')
  print(f"Device tensor is stored on: {tensor.device}")

Device tensor is stored on: cuda:0


## Load data

In [ ]:
# Boundary points from subdomian 1
N_ub   = 250

# Residual points in three subdomains
N_f1   = 500
N_f2   = 200

# Interface points along the two interfaces
N_I1   = 150

# NN architecture in each subdomain
layers1 = [2, 400, 400, 100, 1]
# layers2 = [2, 400, 200, 100, 100, 1]
layers2 = [2, 100, 100, 40, 1]

mu1 = 1
mu2 = 1

Max_iter = 5000

# Load training data (boundary points), residual and interface points from .mat file
# All points are generated in Matlab
# Download file from github
!wget https://github.com/Mushfique12/IGTE/blob/82629c9165acb94b7c2ec77d054b9a3cbd314d06/PINN_Lin_mat.mat?raw=true
# Rename the file
!mv PINN_Lin_mat.mat\?raw\=true PINN_Lin_mat.mat
# Load the file
data = scipy.io.loadmat('/content/PINN_Lin_mat.mat')

--2022-08-17 02:00:53--  https://github.com/Mushfique12/IGTE/blob/82629c9165acb94b7c2ec77d054b9a3cbd314d06/PINN_Lin_mat.mat?raw=true
Resolving github.com (github.com)... 140.82.112.4
Connecting to github.com (github.com)|140.82.112.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://github.com/Mushfique12/IGTE/raw/82629c9165acb94b7c2ec77d054b9a3cbd314d06/PINN_Lin_mat.mat [following]
--2022-08-17 02:00:53--  https://github.com/Mushfique12/IGTE/raw/82629c9165acb94b7c2ec77d054b9a3cbd314d06/PINN_Lin_mat.mat
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/Mushfique12/IGTE/82629c9165acb94b7c2ec77d054b9a3cbd314d06/PINN_Lin_mat.mat [following]
--2022-08-17 02:00:53--  https://raw.githubusercontent.com/Mushfique12/IGTE/82629c9165acb94b7c2ec77d054b9a3cbd314d06/PINN_Lin_mat.mat
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.

# Defining ML Architecture

In [ ]:
R = 1.0
F0 = 1.0

class NNApproximator(nn.Module):
    def __init__(self, num_hidden: int, dim_hidden: int, act=nn.Sigmoid()):

        super().__init__()

        self.layer_in = nn.Linear(2, dim_hidden)
        self.layer_out = nn.Linear(dim_hidden, 1)

        num_middle = num_hidden - 1
        self.middle_layers = nn.ModuleList(
            [nn.Linear(dim_hidden, dim_hidden) for _ in range(num_middle)]
        )
        self.act = act

    def forward(self, x):
        out = self.act(self.layer_in(x))
        for layer in self.middle_layers:
            out = self.act(layer(out))
        return self.act(self.layer_out(out))


def f(nn: NNApproximator, x: torch.Tensor) -> torch.Tensor:
    """Compute the value of the approximate solution from the NN model"""
    return nn(x)


def df(nn: NNApproximator, x: torch.Tensor = None, order: int = 2) -> torch.Tensor:
    """Compute neural network derivative with respect to input features using PyTorch autograd engine"""
    df_value = f(nn, x)
    # print('x.shape', x.shape)
    # output_dim = [int(x.shape[0]), 1]
    # print('output_dim', output_dim)
    # output_ = torch.ones(output_dim)
    # print('output_shape', output_.shape)

    for _ in range(order):
        df_value = torch.autograd.grad(
            df_value,
            x,
            # grad_outputs=torch.ones(output_dim),
            torch.ones_like(df_value),
            # grad_outputs=torch.ones_like(x),
            create_graph=True,
            retain_graph=True,
        )[0]

    return df_value


def compute_loss(
    nn1: NNApproximator, 
    nn2: NNApproximator, 
    epoch : int = 0,
    x1: torch.Tensor = None, 
    x2: torch.Tensor = None, 
    xi: torch.Tensor = None, 
    xb: torch.Tensor = None, 
    ub: torch.Tensor = None,
    ufi: torch.Tensor = None,
    mult: int = 1,
    verbose: bool = False
) -> torch.float:

    """Compute the full loss function as interior loss + boundary loss
    This custom loss function is fully defined with differentiable tensors therefore
    the .backward() method can be applied to it
    """
    multiplier = mult
    interior_loss_1 = torch.sub(df(nn1, x1, order=2), torch.exp(x1))
    interior_loss_2 = torch.sub(df(nn2, x2, order=2), -50)

    u1_i = f(nn1, xi)
    u2_i = f(nn2, xi)
    uAvg_i = torch.mul(torch.add(u1_i, u2_i), 2)

    interface_loss_1 = torch.sub(uAvg_i, u1_i)
    interface_loss_2 = torch.sub(uAvg_i, u2_i)

    actual_interface_loss_1 = torch.sub(ufi, u1_i)
    actual_interface_loss_2 = torch.sub(ufi, u2_i)

    # Interface 1
    interface_1 = df(nn1, xi, order=2)
    interface_2 = df(nn2, xi, order=2)

    # Residual Continuity conditions on the interfaces
    res_loss = (interface_1 - torch.exp(xi)) - (interface_2 - (-50))
    
    # boundary = xb
    # boundary.requires_grad = True
    ub_pred = f(nn1, xb)
    boundary_loss = torch.sub(ub, ub_pred)

    if epoch % 2000 == 0:

      print(f"Loss at {epoch}: {interior_loss_1.pow(2).mean()}, {interface_loss_1.pow(2).mean()}, \
              domain: {interior_loss_2.pow(2).mean()}, {interface_loss_2.pow(2).mean()}, {boundary_loss.pow(2).mean()}")


    loss_1 = \
        interior_loss_1.pow(2).mean() + \
        multiplier * interface_loss_1.pow(2).mean() + \
        multiplier * boundary_loss.pow(2).mean() #+ \
        #multiplier * actual_interface_loss_1.pow(2).mean() #+ \
        #res_loss.pow(2).mean()

    loss_2 = \
        interior_loss_2.pow(2).mean() + \
        multiplier * interface_loss_2.pow(2).mean() #+ \
        #multiplier * actual_interface_loss_2.pow(2).mean() #+ \
        #res_loss.pow(2).mean()

    final_loss = loss_1 + loss_2

    loss_list = [interior_loss_1.pow(2).mean().detach().cpu().numpy(), 
              interface_loss_1.pow(2).mean().detach().cpu().numpy(), 
              interior_loss_2.pow(2).mean().detach().cpu().numpy(), 
              interface_loss_2.pow(2).mean().detach().cpu().numpy(), 
              boundary_loss.pow(2).mean().detach().cpu().numpy()]

    return final_loss, loss_list


def train_model(
    nn1: NNApproximator, 
    nn2: NNApproximator, 
    loss_fn: Callable,
    learning_rate: int = 0.01,
    max_epochs: int = 1000,
):

    loss_evolution_1, loss_evolution_2 = [], []
    interior_loss_1, interface_loss_1, interior_loss_2, interface_loss_2, boundary_loss= [], [], [], [], []

    optimizer = torch.optim.NAdam([
                {'params': nn1.parameters(), 'lr': learning_rate},
                {'params': nn2.parameters(), 'lr': learning_rate}
            ])
    
  
    for epoch in range(max_epochs):

        try:
            loss, loss_list = loss_fn(nn1, nn2, epoch)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            if epoch % 1000 == 0:
                print(f"Epoch: {epoch} - Loss # 1: {float(loss):>7f}")

            loss_evolution_1.append(loss.detach().cpu().numpy())

        except KeyboardInterrupt:
            break

        interior_loss_1.append(loss_list[0]) 
        interface_loss_1.append(loss_list[1]) 
        interior_loss_2.append(loss_list[2]) 
        interface_loss_2.append(loss_list[3]) 
        boundary_loss.append(loss_list[4])

        print('===================================================================')
        print('Here is the Loss Reported', float(loss.detach().cpu().numpy())) 
        print('===================================================================')

        # Reporting to Ray Tune to stop bad performing trials early 
        tune.report(loss=float(loss.detach().cpu().numpy()))

    return [nn1, nn2], np.array(loss_evolution_1), np.array([interior_loss_1, interface_loss_1, interior_loss_2, interface_loss_2, boundary_loss])


def check_gradient(nn: NNApproximator, x: torch.Tensor = None) -> bool:

    eps = 1e-4
    dfdx_fd = (f(nn, x + eps) - f(nn, x - eps)) / (2 * eps)
    dfdx_sample = df(nn, x, order=1)

    return torch.allclose(dfdx_fd.T, dfdx_sample.T, atol=1e-2, rtol=1e-2)




In [ ]:
# nn_approximator_1 = NNApproximator(4, 10)
# nn_approximator_2 = NNApproximator(2, 10)

In [ ]:
# from torchsummary import summary
# print(nn_approximator_1)

In [ ]:
# print(nn_approximator_2)

# Data Preprocessing

In [ ]:
x_f1 = data['x_domain'].flatten()[:,None]
y_f1 = data['y_domain'].flatten()[:,None]
x_f2 = data['x_coil'].flatten()[:,None]
y_f2 = data['y_coil'].flatten()[:,None]

x_i  = data['x_i'].flatten()[:,None]
y_i  = data['y_i'].flatten()[:,None]

x_boundary   = data['x_boundary'].flatten()[:,None]
y_boundary   = data['y_boundary'].flatten()[:,None]

u_boundary  = data['u_boundary'].flatten()[:,None]

u_exact1 = data['u_domain'].flatten()[:,None]
u_exact2 = data['u_coil'].flatten()[:,None]

u_exacti = data['u_i'].flatten()[:,None]

u_exact = np.concatenate([u_exact1, u_exact2, u_exacti, u_boundary])

u_exact1 = np.divide(u_exact1, u_exact.max())

u_exact2 = np.divide(u_exact2, u_exact.max())

u_exacti = np.divide(u_exacti, u_exact.max())

u_boundary = np.divide(u_boundary, u_exact.max())


X_f1 = np.hstack((x_f1.flatten()[:,None], y_f1.flatten()[:,None]))
X_f2 = np.hstack((x_f2.flatten()[:,None], y_f2.flatten()[:,None]))

X_fi = np.hstack((x_i.flatten()[:,None], y_i.flatten()[:,None]))

X_ub = np.hstack((x_boundary.flatten()[:,None], y_boundary.flatten()[:,None]))

# Points in the whole  domain
# x_total =  data['x_total'].flatten()[:,None] 
# y_total =  data['y_total'].flatten()[:,None]

# X_star1 = np.hstack((x_f1.flatten()[:,None], y_f1.flatten()[:,None]))
# X_star2 = np.hstack((x_f2.flatten()[:,None], y_f2.flatten()[:,None]))

# Randomly select the residual points from sub-domains
idx1 = np.random.choice(X_f1.shape[0], N_f1, replace=False)    
X_f1_train = np.float32(X_f1[idx1,:])

idx2 = np.random.choice(X_f2.shape[0], N_f2, replace=False)    
X_f2_train = np.float32(X_f2[idx2,:])

# Randomly select boundary points
idx4 = np.random.choice(X_ub.shape[0], N_ub, replace=False)
X_ub_train = np.float32(X_ub[idx4,:])
ub_train   = np.float32(u_boundary[idx4,:])

# Randomly select the interface points along two interfaces
idxi1 = np.random.choice(X_fi.shape[0], N_I1, replace=False)    
X_fi_train = np.float32(X_fi[idxi1,:])
ub_fi1_train = np.float32(u_exacti[idxi1,:])

In [ ]:
type(X_f1_train[1, 0])

numpy.float32

# Plotting Each losses


In [ ]:
def plot_losses():
  fig = plt.figure(figsize=(10, 7))

  plt.plot(loss_array[0, :])
  plt.title('interior_loss_1')
  plt.show()

  fig = plt.figure(figsize=(12, 8))

  plt.plot(loss_array[1, :])
  plt.title('interface_loss_1')
  plt.show()

  fig = plt.figure(figsize=(12, 8))

  plt.plot(loss_array[2, :])
  plt.title('interior_loss_2')
  plt.show()

  fig = plt.figure(figsize=(12, 8))

  plt.plot(loss_array[3, :])
  plt.title('interface_loss_2')

  plt.show()

  fig = plt.figure(figsize=(12, 8))

  plt.plot(loss_array[4, :])
  plt.title('boundary_loss')

  plt.show()

## Predict Output


In [ ]:
# Solution prediction

def predict():
  nn1_trained = nn_approximator_trained[0].cpu()
  nn2_trained = nn_approximator_trained[1].cpu()

  u_pred1 = nn1_trained(torch.tensor(np.float32(X_f1))).detach().numpy()
  u_pred2 = nn2_trained(torch.tensor(np.float32(X_f2))).detach().numpy()

  u_pred_i1 = nn1_trained(torch.tensor(np.float32(X_fi))).detach().numpy()
  u_pred_i2 = nn2_trained(torch.tensor(np.float32(X_fi))).detach().numpy()

  u_pred_b = nn2_trained(torch.tensor(np.float32(X_ub))).detach().numpy()

  u_pred = np.concatenate([u_pred1, u_pred2, (u_pred_i1 + u_pred_i2)/2, u_pred_b])

In [ ]:
# u_exact1 = data['u_domain'].flatten()[:,None]
# u_exact2 = data['u_coil'].flatten()[:,None]

# u_exacti = data['u_i'].flatten()[:,None]

# u_exact = np.concatenate([u_exact1, u_exact2, u_exacti, u_boundary])


In [ ]:
# u_exact1 = np.divide(u_exact1, u_exact.max())
# print(u_exact1.max(), u_exact1.min())

In [ ]:
# u_exact2 = np.divide(u_exact2, u_exact.max())
# print(u_exact2.max(), u_exact2.min())

In [ ]:
# u_exacti = np.divide(u_exacti, u_exact.max())
# print(u_exacti.max(), u_exacti.min())

In [ ]:
# u_boundary = np.divide(u_boundary, u_exact.max())
# print(u_boundary.max(), u_boundary.min())

In [ ]:
# u_exact = np.concatenate([u_exact1, u_exact2, u_exacti, u_boundary])


## Max values

u max: 2.2

u min: 0


In [ ]:
# x_tot = np.concatenate([x_f1, x_f2, x_i, x_boundary])
# y_tot = np.concatenate([y_f1, y_f2, y_i, y_boundary])
# print(x_tot.shape, y_tot.shape, u_pred.shape, u_exact.shape)

# Plotting Contours


In [ ]:
import matplotlib.pyplot as plt
import matplotlib.tri as tri
import numpy as np

def plot_contour():
  np.random.seed(19680801)
  npts = 200
  ngridx = 100
  ngridy = 200
  x = x_tot.squeeze()
  y = y_tot.squeeze()
  up = u_pred.squeeze()
  uex = u_exact.squeeze()

  fig, (ax1, ax2) = plt.subplots(1,2,figsize=(26,10))
  vmin=0
  vmax=2.2

  ## True Exact
  # ax2.tricontour(x, y, z, levels=20, linewidths=0.5, colors='k')
  cntr1 = ax1.tricontourf(x, y, uex, levels=20, vmin=vmin, vmax=vmax, cmap="jet")

  fig.colorbar(cntr1, ax=ax1)
  # ax2.plot(x, y, 'ko', ms=3)
  ax1.set(xlim=(-1, 1), ylim=(-1, 1))
  ax1.set_title('Exact')


  ## Predictions
  # ax2.tricontour(x, y, z, levels=20, linewidths=0.5, colors='k')
  cntr2 = ax2.tricontourf(x, y, up, levels=20, vmin=vmin, vmax=vmax, cmap="jet")

  fig.colorbar(cntr2, ax=ax2)
  # ax2.plot(x, y, 'ko', ms=3)
  ax2.set(xlim=(-1, 1), ylim=(-1, 1))
  ax2.set_title('Prediction')

  plt.subplots_adjust(hspace=0.5)
  # plt.savefig('exact_pred_TL_20points.png', dpi=400)
  plt.show()

## Train Pinns

In [ ]:
from functools import partial
import os

# domain = [0.0, 1.0]
# x = torch.linspace(domain[0], domain[1], steps=10, requires_grad=True)
# x = x.reshape(x.shape[0], 1)

def train_cifar(config, checkpoint_dir=None):
  nn_approximator_1 = NNApproximator(config["l1"], config["l2"])
  nn_approximator_2 = NNApproximator(config["l1"], config["l2"])
  
  device = "cpu"
  if torch.cuda.is_available():
      device = "cuda:0"
      if torch.cuda.device_count() > 1:
          nn_approximator_1 = nn.DataParallel(nn_approximator_1)
          nn_approximator_2 = nn.DataParallel(nn_approximator_2)
  nn_approximator_1.to(device)
  nn_approximator_2.to(device)

  x1 = torch.tensor(X_f1_train, requires_grad=True)
  x2 = torch.tensor(X_f2_train, requires_grad=True) 
  xi = torch.tensor(X_fi_train, requires_grad=True) 
  xb = torch.tensor(X_ub_train, requires_grad=True)
  ub = torch.tensor(ub_train)
  ub = torch.zeros_like(ub)
  ufi = torch.tensor(ub_fi1_train)

  x1 = x1.to(device)
  x2 = x2.to(device)
  xi = xi.to(device)
  xb = xb.to(device)
  ub = ub.to(device)
  ufi = ufi.to(device)

  #rint(f"Device x1 is stored on: {x1.device}")
  #print(f"Device x2 is stored on: {x2.device}")

  
  # assert check_gradient(nn_approximator, x)

  # f_initial = f(nn_approximator, x)
  # ax.plot(x.detach().numpy(), f_initial.detach().numpy(), label="Initial NN solution")

  # if checkpoint_dir:
  #   model_state = torch.load(
  #       os.path.join(checkpoint_dir, "checkpoint"))
  #   net.load_state_dict(model_state)
    # optimizer.load_state_dict(optimizer_state)

  # train the PINN
  loss_fn = partial(compute_loss,     
                    x1 = x1, 
                    x2 = x2, 
                    xi = xi, 
                    xb = xb, 
                    ub = ub,
                    ufi = ufi,
                    mult = 20,
                    verbose=True)

  nn_approximator_trained, loss_evolutions, loss_array = train_model(
                                                                    nn_approximator_1,
                                                                    nn_approximator_2,
                                                                    loss_fn=loss_fn, 
                                                                    learning_rate=0.008, 
                                                                    max_epochs=5000
                                                                  )

  print("Finished Training")

  # x_eval = torch.linspace(domain[0], domain[1], steps=100).reshape(-1, 1)

# Hyper-Parameter Tuning

In [ ]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 16482933586364995425
 xla_global_id: -1, name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 13914341376
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 12576702658174337477
 physical_device_desc: "device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5"
 xla_global_id: 416903419]

In [23]:
max_num_epochs = 30000
num_samples = 30
gpus_per_trial = 1

config = {
        "l1": tune.sample_from(lambda _: np.random.randint(2, 9)),
        "l2": tune.sample_from(lambda _: 2 ** np.random.randint(4, 9)),
        "lr": tune.loguniform(1e-4, 1e-1),
        "multiplier": tune.grid_search([20, 40, 60, 90]),
        "epochs": tune.grid_search([30000, 60000])
    }

scheduler = ASHAScheduler(
        metric="loss",
        mode="min",
        max_t=max_num_epochs,
        grace_period=1,
        reduction_factor=2)

reporter = CLIReporter(
    # parameter_columns=["l1", "l2", "lr", "batch_size"],
    metric_columns=["loss", "training_iteration"])

result = tune.run(
    partial(train_cifar),
    resources_per_trial={"cpu": 1, "gpu": gpus_per_trial},
    config=config,
    num_samples=num_samples,
    scheduler=scheduler,
    progress_reporter=reporter)

best_trial = result.get_best_trial(metric = "loss", mode = "min")
print("Best trial config: {}".format(best_trial.config))
print("Best trial final loss: {}".format(best_trial.last_result["loss"]))

Streaming output truncated to the last 5000 lines.
  iterations_since_restore: 1
  loss: 2642.503173828125
  node_ip: 172.28.0.2
  pid: 18858
  time_since_restore: 2.5174825191497803
  time_this_iter_s: 2.5174825191497803
  time_total_s: 2.5174825191497803
  timestamp: 1660704779
  timesteps_since_restore: 0
  training_iteration: 1
  trial_id: 085be_00213
  warmup_time: 0.004410505294799805
  
(func pid=18858) Loss at 0: 1.6387672424316406, 3.74178147315979,               domain: 2500.000244140625, 3.0346059799194336, 0.26682472229003906
(func pid=18858) Epoch: 0 - Loss # 1: 2642.503174
(func pid=18858) ===================================================================
(func pid=18858) Here is the Loss Reported 2642.503173828125
(func pid=18858) ===================================================================
== Status ==
Current time: 2022-08-17 02:53:00 (running for 00:26:17.24)
Memory usage on this node: 2.6/12.7 GiB
Using AsyncHyperBand: num_stopped=212
Bracket: Iter 16384.000:

2022-08-17 02:55:43,523	INFO tune.py:748 -- Total run time: 1740.65 seconds (1740.26 seconds for the tuning loop).


== Status ==
Current time: 2022-08-17 02:55:43 (running for 00:29:00.32)
Memory usage on this node: 3.6/12.7 GiB
Using AsyncHyperBand: num_stopped=238
Bracket: Iter 16384.000: None | Iter 8192.000: None | Iter 4096.000: -108.6067008972168 | Iter 2048.000: -1370.0469131469727 | Iter 1024.000: -2500.398681640625 | Iter 512.000: -2501.638671875 | Iter 256.000: -2501.638916015625 | Iter 128.000: -2501.639892578125 | Iter 64.000: -2501.64013671875 | Iter 32.000: -2501.640869140625 | Iter 16.000: -2501.646728515625 | Iter 8.000: -2501.699951171875 | Iter 4.000: -2504.0679931640625 | Iter 2.000: -2537.97021484375 | Iter 1.000: -2595.98974609375
Resources requested: 0/2 CPUs, 0/1 GPUs, 0.0/7.31 GiB heap, 0.0/3.65 GiB objects (0.0/1.0 accelerator_type:T4)
Result logdir: /root/ray_results/train_cifar_2022-08-17_02-26-42
Number of trials: 240/240 (240 TERMINATED)
+-------------------------+------------+------------------+----------+------+------+-------------+--------------+-----------+----------

In [ ]:
best_trial = result.get_best_trial("loss", "min", "last")
print(best_trial)

train_cifar_e2b62_00000


In [ ]:
plot_losses()

## Rough

In [ ]:
# # numeric solution
# def logistic_eq_fn(x, y):
#     return R * x * (1 - x)

# numeric_solution = solve_ivp(
#     logistic_eq_fn, domain, [F0], t_eval=x_eval.squeeze().detach().numpy()
# )

# # plotting
# fig, ax = plt.subplots()

# f_final_training = f(nn_approximator_trained, x)
# f_final = f(nn_approximator_trained, x_eval)

# ax.scatter(x.detach().numpy(), f_final_training.detach().numpy(), label="Training points", color="red")
# ax.plot(x_eval.detach().numpy(), f_final.detach().numpy(), label="NN final solution")
# ax.plot(
#     x_eval.detach().numpy(),
#     numeric_solution.y.T,
#     label=f"Analytic solution",
#     color="green",
#     alpha=0.75,
# )
# ax.set(title="Logistic equation solved with NNs", xlabel="t", ylabel="f(t)")
# ax.legend()

# fig, ax = plt.subplots()
# ax.semilogy(loss_evolution)
# ax.set(title="Loss evolution", xlabel="# epochs", ylabel="Loss")
# ax.legend()

# plt.show()

NameError: ignored